In [8]:
import pandas as pd

In [5]:
path_keywords_list = './keywords_list.txt'
path_featurevec_csv = './XSDHandler.csv'

In [10]:
with open(path_keywords_list, 'r') as f:
    keywords_list = f.read().splitlines()

print(keywords_list)

['create', 'object', 'cache', 'uri', 'standalone', 'encoding', 'identifier', 'user', 'error', 'content', 'parameter', 'subset', 'global', 'component']


In [21]:
df = pd.read_csv(path_featurevec_csv)
methods = df['method_name'].values
methods

array(['addGlobalAttributeGroupDecls', 'addGlobalGroupDecls', 'reset',
       'getGlobalDecl', 'element2Locator', 'findXSDocumentForDecl',
       'prepareForTraverse', 'schemaDocument2SystemId',
       'expandComponents', 'addGlobalTypeDecls',
       'expandRelatedComplexTypeComponents', 'addGlobalAttributeDecls',
       'findDependentNamespaces', 'resolveSchema', 'null2EmptyString',
       'getIDRegistry', 'expandRelatedElementComponents',
       'expandRelatedComponents', 'addGlobalTypeDecl',
       'expandRelatedAttributeComponents', 'setDeclPool',
       'getGlobalDeclFromGrammar', 'reportSchemaWarning',
       'resolveKeyRefs', 'getGlobalElementDecl', 'setDVFactory',
       'createTraversers', 'traverseLocalElements',
       'updateImportDependencies', 'addGlobalGroupDecl',
       'addGlobalComponents', 'addGrammars', 'canAddComponents',
       'createGrammarFrom', 'addNewGrammarComponents',
       'needReportTNSError', 'addIDConstraintDecl', 'fillInLocalElemInfo',
       'nonAnno

In [22]:
ground_truths = {}
for method in methods:
    for keyword in keywords_list:
        if keyword in method.lower():
            if keyword not in ground_truths:
                ground_truths[keyword] = []
            ground_truths[keyword].append(method)
            break
    else:
        if 'none' not in ground_truths:
            ground_truths['none'] = []
        ground_truths['none'].append(method)

In [24]:
print(ground_truths.keys())

dict_keys(['global', 'none', 'component', 'create', 'error', 'content'])


In [48]:
df_to_csv = df['method_name'].to_frame()
df_to_csv['cluster_id'] = -1
cols = df_to_csv.columns.tolist()
df_to_csv = df_to_csv[cols[-1:] + cols[:-1]]
for i, keyword in enumerate(ground_truths):
    for method in ground_truths[keyword]:
        df_to_csv.loc[df_to_csv['method_name'] == method, 'cluster_id'] = i
        
    

In [49]:
# sanity check
set1 = set(df_to_csv.loc[df_to_csv['cluster_id'] == 0]['method_name'].values.tolist())
set2 = set(ground_truths['global'])
print(set1==set2)

True


In [51]:
df_to_csv.sort_values(by='cluster_id', inplace=True)
file_name = path_featurevec_csv.split('/')[-1].split('.')[0]
file_name = 'ground_truth'+file_name+'_.csv'
df_to_csv.to_csv(file_name, index=False)   